# Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importing libraries for web scraping

In [2]:
from bs4 import BeautifulSoup
import csv

Getting all the links from csv file

In [3]:
with open('Links.csv') as f:
    reader = csv.reader(f)
    links = [row[0] for row in reader]

In [4]:
from selenium import webdriver
driver = webdriver.Chrome('C:\wedrivers\chromedriver.exe')

In [5]:
reviews = []
for link in links:
    response = driver.get(link)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source,'html.parser')
    i = soup.find_all('div',{'class':'reviewSnippetCard'})
    for j in i:
        review = j.find('p',{'class':'snippetSummary'}).text
        rating = int(j.find('span',{'class':'stars-icon-star-on'})['style'][6:-2])/20
        reviews.append([review,rating])
        

In [6]:
dataset = pd.DataFrame(reviews, columns=['Reviews','Rating'])

In [7]:
dataset.to_csv('reviews.csv',index=False)

# Applying Natural Language Processing

In [8]:
import re
import nltk

In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [10]:
ps = PorterStemmer()

Preprocessing data

In [11]:
processed_review = []

for i in range(len(dataset)):
    review = re.sub('@[\w]*', ' ',dataset['Reviews'][i])
    review = re.sub('^a-zA-Z#', ' ',review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    processed_review.append(review)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 3000)

In [13]:
X = dataset['Reviews'].values
y = dataset['Rating'].values

In [14]:
X = cv.fit_transform(processed_review).toarray()

In [15]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [16]:
for i in range(len(y)):
    if(y[i] >=3.5):
        y[i] = 1 #postive
        
    else:
        y[i] = 0 #negative

# Splitting into training and test dataset

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y ,test_size = 0.2)

# Applying Naive Bayes Classifier

In [18]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [19]:
nb.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [20]:
nb.score(X_train,y_train)

0.9791666666666666

In [21]:
nb.score(X_test,y_test)

0.875